In [ ]:
from db_inference import simple_sql_db

In [ ]:
sql_db = simple_sql_db.SimpleSqlDb()

LIMIT = 100000000 # 100M

In [ ]:
%%time

cur = sql_db.conn.cursor()

edge_csv_fpath = "/GeneGraphDB/data/test_edges.csv"
node_csv_fpath = "/GeneGraphDB/data/test_nodes.csv"

unique_p100s = set()


for row in cur.execute(f"SELECT * FROM prot2protwindow LIMIT {LIMIT}"):
# for row in cur.execute("SELECT * FROM prot2protwindow LIMIT 1000"):
    unique_p100s.add(row['p1hash'])
    unique_p100s.add(row['p2hash'])

print(len(unique_p100s))

In [ ]:
%%time
p100_to_idx = {p_hash: idx+1 for idx, p_hash in enumerate(unique_p100s)}

In [ ]:
%%time

with open(node_csv_fpath, 'w') as node_fp:

    node_fp.write("id:ID(P100),p100\n")

    for node_hash, node_idx in p100_to_idx.items():
        node_fp.write(",".join((str(node_idx), node_hash))+"\n")
    

print(f"wrote {node_csv_fpath}")

In [ ]:
%%time

with open(edge_csv_fpath, 'w') as edge_fp:

    edge_fp.write(":START_ID(P100),:END_ID(P100)\n")

    for row in cur.execute(f"SELECT * FROM prot2protwindow LIMIT {LIMIT}"):
        start_idx = p100_to_idx[row['p1hash']]
        end_idx = p100_to_idx[row['p2hash']]
        edge_fp.write(f"{start_idx},{end_idx}\n")
    
print(f"wrote {edge_csv_fpath}")

In [ ]:
import os
os.listdir('/GeneGraphDB/data')